# Use DynamoDB as a Cache Backend

NOTE: this solution is based on [pynamodb_mate](https://github.com/MacHu-GWU/pynamodb_mate-project) Python library.

A lots of developer love Redis. However, redis is not a serverless solution, and you need to manage the server (or cluster). **DynamoDB naturally is a serverless, distributive, Key-Value database that has very high read / write throughput. It is a good choice to use DynamoDB as a cache without technique overhead**.

⭐ Benefit:

- There's no server to manage.
- DynamoDB has a latency around 20ms per request.
- DynamoDB cache backend can be created in 5 seconds.
- DynamoDB has pay-as-you-go pricing model, you only pay for what you use.
- DynamoDB automatically scales up and down to adapt your traffic.
- Unlike other local cache solutions, it is on cloud and has access management out-of-the-box.


## Declare a Cache Backend

Cache is nothing but just a key-value store. It uses binary as the storage protocol to store your data. You have to explicitly tell the cache how you convert your data to binary or how you recover your data from binary.

In [14]:
import dataclasses
import time
import pickle
import pynamodb_mate as pm

In [4]:
# assume that this is your data
@dataclasses.dataclass
class PersonProfile:
    name: str
    ssn: str

# pm.patterns.cache.DynamoDBBackend[PersonProfile] tells the cache that your data type is PersonProfile
# so you will have type hint automatically
class MyCache(pm.patterns.cache.DynamoDBBackend[PersonProfile]):
    def serialize(self, value: PersonProfile) -> bytes:
        return pickle.dumps(value)

    def deserialize(self, value: bytes) -> PersonProfile:
        return pickle.loads(value)

cache = MyCache(table_name="pynamodb-example-cache")

**THAT's IT**, by default it uses pay as you go pricing model.

## Use ``Cache.set`` and ``Cache.get``

In [11]:
key = "alice"

# cache not hit
print(cache.get(key))

PersonProfile(name='Alice', ssn='123-45-6789')


In [12]:
cache.set(key, PersonProfile(name="Alice", ssn="123-45-6789"))

In [13]:
cache.get(key)

PersonProfile(name='Alice', ssn='123-45-6789')

## Use Time-to-live (TTL)

In [15]:
key = "bob"

# set the key-value with TTL
cache.set(key, PersonProfile(name="Bob", ssn="123-45-6789"), expire=1)

print("we try to get the value immediately, the cache should be hit:")
print(cache.get(key))

print("after wait for 2 seconds, the cache should be expired:")
time.sleep(2)
print(cache.get(key))

we try to get the value immediately, the cache should be hit:
PersonProfile(name='Bob', ssn='123-45-6789')
after wait for 2 seconds, the cache should be expired:
None
